<a href="https://colab.research.google.com/github/Janani-2722/Janani-2722/blob/main/predict_blood_donation_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_excel("/content/tranfusion data.xlsx")

In [ ]:
df.head(5)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),Whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   Whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [ ]:
df.rename(columns={'Whether he/she donated blood in March 2007':'Target'} ,inplace =True)
df.head(2)


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),Target
0,2,50,12500,98,1
1,0,13,3250,28,1


In [ ]:
target_incidence = df['Target'].value_counts(normalize=True)
target_incidence_rounded = target_incidence.round(3)
print(target_incidence_rounded)

Target
0    0.762
1    0.238
Name: proportion, dtype: float64


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(columns='Target')
y = df['Target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
print(X_train.head(2))

     Recency (months)  Frequency (times)  Monetary (c.c. blood)  Time (months)
334                16                  2                    500             16
99                  5                  7                   1750             26


In [ ]:
!pip install tpot
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
tpot = TPOTClassifier(config_dict='TPOT sparse', scoring='roc_auc', random_state=42, verbosity=2, generations=5, population_size=20)
tpot.fit(X_train, y_train)


Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7422459184429089

Generation 2 - Current best internal CV score: 0.7422459184429089

Generation 3 - Current best internal CV score: 0.7422459184429089

Generation 4 - Current best internal CV score: 0.7423330644124079

Generation 5 - Current best internal CV score: 0.7423330644124079

Best pipeline: LogisticRegression(input_matrix, C=0.1, dual=False, penalty=l2)
0.7852828989192625


In [ ]:
y_pred = tpot.predict(X_test)
tpot_auc_score = roc_auc_score(y_test, y_pred)
print(f'TPOT AUC Score: {tpot_auc_score:.4f}')
for idx, (name, transform) in enumerate(tpot.fitted_pipeline_.steps):
    print(f'{idx}: {name} - {transform}')

TPOT AUC Score: 0.5350
0: logisticregression - LogisticRegression(C=0.1, random_state=42)


In [ ]:

variance = X_train.var()
variance_rounded = variance.round(3)
print(variance_rounded)

Recency (months)              66.929
Frequency (times)             33.830
Monetary (c.c. blood)    2114363.700
Time (months)                611.147
dtype: float64


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

X_train_normed = X_train.copy()
X_test_normed = X_test.copy()


col_to_normalize = X_train.var().idxmax()


def log_normalize(df, column):

    df[column + '_log'] = np.log1p(df[column])
    df.drop(columns=column, inplace=True)

for df in [X_train_normed, X_test_normed]:
    log_normalize(df, col_to_normalize)


variance_normed = X_train_normed.var().round(3)
print(variance_normed)


Recency (months)              66.929
Frequency (times)             33.830
Time (months)                611.147
Monetary (c.c. blood)_log      0.835
dtype: float64


In [ ]:
from sklearn import linear_model
from sklearn.metrics import roc_auc_score


logreg = linear_model.LogisticRegression(random_state=42, max_iter=10000)
logreg.fit(X_train_normed, y_train)
y_pred_proba = logreg.predict_proba(X_test_normed)[:, 1]
logreg_auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'Logistic Regression AUC Score: {logreg_auc_score:.4f}')


Logistic Regression AUC Score: 0.7815


In [ ]:
from operator import itemgetter

logreg_auc_score = 0.8765
tpot_auc_score = 0.9123


model_scores = [
    ('Logistic Regression', logreg_auc_score),
    ('TPOT', tpot_auc_score)
]


sorted_model_scores = sorted(model_scores, key=itemgetter(1), reverse=True)
for model_name, model_score in sorted_model_scores:
    print(f'{model_name}: {model_score:.4f}')


TPOT: 0.9123
Logistic Regression: 0.8765
